In [2]:
!pip3 install transformers
!pip3 install datasets
!pip3 install rouge_score
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [4]:
model_checkpoint = "t5-small"

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [6]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("eli5")
metric = load_metric("rouge")

Downloading:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/286M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/18.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

Dataset eli5 downloaded and prepared to /root/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa. Subsequent calls will reuse this data.


  0%|          | 0/9 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [7]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b"]:
  qprefix = "question:"
  cprefix = "support:"
else:
  qprefix = ""
  cprefix = ""

In [8]:
inputs = [qprefix + " " + doc[0] + " " + cprefix + " " + doc[1] for doc in zip(raw_datasets['train_eli5'][:2]["title"], raw_datasets['train_eli5'][:2]['selftext'])]

In [16]:
max_input_length = 128
max_target_length = 512

def preprocess_data(examples):
  inputs = [qprefix + " " + doc[0] + " " + cprefix + " " + doc[1] for doc in zip(examples["title"], examples['selftext'])]
  # print(inputs)
  model_inputs = tokenizer(inputs, max_length = max_input_length, truncation = True)

  with tokenizer.as_target_tokenizer():
    texts = [x['text'][0] for x in examples['answers']]
    # print(texts)
    # labels = tokenizer(texts, max_length = max_target_length, truncation = True)
    labels = tokenizer(["[SEP]".join(p['text']) for p in examples['answers']], max_length = max_target_length, truncation = True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [17]:
preprocess_data(raw_datasets['train_eli5'][:2])

{'input_ids': [[822, 10, 16, 3370, 125, 7, 8, 500, 13, 3, 26281, 8, 166, 192, 4805, 28, 3, 9, 10505, 3, 18, 95, 8, 2214, 3, 18, 59, 1646, 10505, 4805, 3, 23, 129, 273, 380, 10, 1], [822, 10, 1615, 33, 315, 3, 3276, 7, 41, 60, 122, 4885, 3, 2, 2076, 3, 2, 3331, 61, 13, 1807, 31, 1596, 966, 373, 335, 3151, 7, 315, 58, 380, 10, 27, 31, 162, 4944, 24, 8, 1750, 16, 594, 344, 1646, 1807, 11, 2076, 5517, 6, 11, 344, 2076, 5517, 11, 3331, 6, 19, 966, 373, 335, 3151, 7, 5, 100, 1330, 12, 1520, 1176, 150, 1052, 125, 8, 594, 21, 1646, 1807, 5, 100, 744, 31, 17, 1727, 12, 143, 1254, 6, 38, 8, 1750, 344, 3771, 11, 3771, 5, 1714, 11, 8, 1750, 344, 7514, 11, 7514, 5, 1714, 3, 87, 6191, 33, 7385, 1427, 182, 315, 5, 27, 7, 48, 131, 46, 3, 26968, 8346, 24, 23248, 7, 12874, 13, 3, 9, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [18]:
tokenized_dataset = raw_datasets.map(preprocess_data, batched = True)

  0%|          | 0/273 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/132 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/99 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [19]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [28]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-eli5",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [30]:
import nltk
nltk.download('punkt')
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train_eli5"],
    eval_dataset=tokenized_dataset["validation_eli5"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/t5-small-finetuned-eli5 is already a clone of https://huggingface.co/Sandipan1994/t5-small-finetuned-eli5. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend


In [32]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: answers, subreddit, selftext, document, selftext_urls, title_urls, title, q_id, answers_urls. If answers, subreddit, selftext, document, selftext_urls, title_urls, title, q_id, answers_urls are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 272634
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 17040


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.964800,3.754533,9.697200,1.830300,7.813200,8.996400,18.998200


Saving model checkpoint to t5-small-finetuned-eli5/checkpoint-500
Configuration saved in t5-small-finetuned-eli5/checkpoint-500/config.json
Model weights saved in t5-small-finetuned-eli5/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-eli5/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-eli5/checkpoint-500/special_tokens_map.json
tokenizer config file saved in t5-small-finetuned-eli5/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-eli5/special_tokens_map.json
Saving model checkpoint to t5-small-finetuned-eli5/checkpoint-1000
Configuration saved in t5-small-finetuned-eli5/checkpoint-1000/config.json
Model weights saved in t5-small-finetuned-eli5/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-eli5/checkpoint-1000/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-eli5/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to t5-sma

TrainOutput(global_step=17040, training_loss=4.018481892151452, metrics={'train_runtime': 9112.8307, 'train_samples_per_second': 29.918, 'train_steps_per_second': 1.87, 'total_flos': 8503982165065728.0, 'train_loss': 4.018481892151452, 'epoch': 1.0})

In [34]:
metrics = trainer.evaluate(tokenized_dataset['test_eli5'])

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: answers, subreddit, selftext, document, selftext_urls, title_urls, title, q_id, answers_urls. If answers, subreddit, selftext, document, selftext_urls, title_urls, title, q_id, answers_urls are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 24512
  Batch size = 16


In [36]:
trainer.push_to_hub()

Saving model checkpoint to t5-small-finetuned-eli5
Configuration saved in t5-small-finetuned-eli5/config.json
Model weights saved in t5-small-finetuned-eli5/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-eli5/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-eli5/special_tokens_map.json


Upload file runs/Sep26_03-36-13_87070f92ad8d/events.out.tfevents.1664174163.87070f92ad8d.325.4: 100%|#########…

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/Sandipan1994/t5-small-finetuned-eli5
   57e9a69..1b403ff  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/Sandipan1994/t5-small-finetuned-eli5
   57e9a69..1b403ff  main -> main

To https://huggingface.co/Sandipan1994/t5-small-finetuned-eli5
   1b403ff..296728c  main -> main

   1b403ff..296728c  main -> main



'https://huggingface.co/Sandipan1994/t5-small-finetuned-eli5/commit/1b403ffb02be0c82d1de5b24097a3eef9b41d63a'

In [35]:
#### Test Metrics ######

metrics 

{'eval_loss': 3.7516181468963623,
 'eval_rouge1': 9.5595,
 'eval_rouge2': 1.7879,
 'eval_rougeL': 7.7084,
 'eval_rougeLsum': 8.8886,
 'eval_gen_len': 18.9998,
 'eval_runtime': 878.1995,
 'eval_samples_per_second': 27.912,
 'eval_steps_per_second': 1.744,
 'epoch': 1.0}